In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import numpy as np

from sklearn.naive_bayes import _BaseDiscreteNB, MultinomialNB
# import modules/downloads

import pandas as pd
import re

from sklearn.metrics import v_measure_score

from sklearn.metrics import f1_score

In [2]:
import pandas as pd

extra_paragraphs = pd.read_csv("./tidy_text_data.csv")
extra_paragraphs = extra_paragraphs[:1250]

train = pd.read_csv("./train.txt", sep=";")
train.columns = ["text","sentiment"]
test = pd.read_csv("./test.txt", sep=";")
test.columns = ["text","sentiment"]

train_text = train.text
extra_paragraphs_text = extra_paragraphs.text_new

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=5000, min_df=3, binary=True, 
                             stop_words="english")
X_supervised = vectorizer.fit_transform(train_text)
X_unsupervised = vectorizer.transform(extra_paragraphs_text)

X_supervised.shape, X_unsupervised.shape

X_test = vectorizer.transform(test.text)
test_sentiment = test.sentiment

In [3]:
import numpy as np
combined_X = np.concatenate((X_supervised.toarray(), X_unsupervised.toarray()), axis = 0)
combined_X.shape

(17249, 4885)

In [4]:
mask_1 = np.ones(X_supervised.shape[0], dtype=int)
mask_2 = np.zeros(X_unsupervised.shape[0], dtype=int)

mask = np.concatenate((mask_1,mask_2), axis =0)

In [5]:
# function to get unique values
def unique(list1):
 
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

topics = unique(train['sentiment'])
y_train = np.array([topics.index(k) for k in train['sentiment']],dtype=int)
y_test = np.array([topics.index(k) for k in test_sentiment],dtype=int)
combined_Y = np.concatenate((y_train, np.zeros(X_unsupervised.shape[0], dtype=int)), axis=0)
print(len(unique(combined_Y)))
# y_test.shape

6


In [6]:
from sklearn.naive_bayes import _BaseDiscreteNB, MultinomialNB

class SoftMultinomialNB(MultinomialNB):

    # Note y = numpy array of size n x k containing 
    # for each example n, the probability of it belonging to class k.
    
    def fit(self, X, y, sample_weight=None):        
      _, n_features = X.shape
      self.n_features_in_ = n_features 

      Y = y
      self.classes_ = np.arange(Y.shape[1])

      if sample_weight is not None:
          Y = Y.astype(np.float64, copy=False)
          sample_weight = np.asarray(sample_weight)
          sample_weight = np.atleast_2d(sample_weight)
          Y *= check_array(sample_weight).T

      class_prior = self.class_prior

      # Count raw events from data before updating the class log prior
      # and feature log probas
      n_effective_classes = Y.shape[1]

      self._init_counters(n_effective_classes, n_features)
      self._count(X, Y)
      alpha = self._check_alpha()
      self._update_feature_log_prob(alpha)
      self._update_class_log_prior(class_prior=class_prior)
      return self

In [7]:
def semisupervised_EM(init_model,k, X, y_true, mask, max_iters=200, soft=False,
                      return_model=False):
    '''
    Implements semi-supervised Hard or Soft EM model.
        - init_model is a function for initializing the SoftMultinomialNB model
            (hint: you are going to want to use this parameter later in the
             homework to initialize in a different model)
        - X is an (n x d) numpy array
        - y_true - 0/1 class labels defined previously.
        - mask - an array of size n containing True or False indicating if you
            are to use the true labels of that example or not.
        - soft - a boolean variable, which if true, performs soft EM and
            otherwise performs hard EM
        - return_model - specifies whether or not to return the NB classifier
    
    Return a list of size n, containing the class label (between 0 and k-1) of
    each item.
    '''
    n, d = X.shape
    X.shape
    temp = np.zeros((len(y_true),k))
    for i in range(temp.shape[0]):
        temp[i,y_true[i]] = 1
    
    m = init_model()
    x_train = X[mask]
    # y_train = y_true[mask]
    y_train_mat = temp[mask]
    m.fit(x_train, y_train_mat)
    y_hat = None
    for _ in tqdm(range(max_iters)):
        # E Step: predict the latent variables (y_hat) using the model
        newy_hat = m.predict_proba(X)
        if not soft:
            newy_hat = np.floor(0.5 + newy_hat)
        # converged, return model
        if (newy_hat == y_hat).all():
            if return_model:
                return newy_hat, m
            else: 
                return newy_hat
        y_hat = newy_hat
        #M Step: retrain the model using the latent variables
        y_hat[mask] = y_train_mat
        m.fit(X, y_hat)

    y_hat = m.predict(X)
    if return_model:
        return y_hat, m
    else:
        return y_hat

In [8]:
yhat, m = semisupervised_EM(lambda: SoftMultinomialNB(), 6, combined_X, combined_Y, mask, return_model=True, soft=True)

AttributeError: 'SoftMultinomialNB' object has no attribute 'super'

In [ ]:
def breakdown_clusters_by_ground_truth(y_true, cluster_labels, display=True) :
    """
    If display = True, 
        display counts of documents of each topic in each of the generated
        clusters and plot the same.
        
    Returns v-measure of cluster labels with respect to ground truth
    """
    
    assert len(y_true) == len(cluster_labels), "Length of Target and cluster labels don't match"
    df = pd.DataFrame({'topic' : y_true, 'cluster' : cluster_labels})
    counts = df.groupby(['topic', 'cluster']).size().reset_index().rename(columns={0:'count'})
    if display :
        print(counts)
        sns.barplot(x="topic", y="count", hue="cluster", data=counts)
    return v_measure_score(y_true, cluster_labels)

from sklearn.metrics import f1_score

prediction = m.predict(X_supervised)
print(prediction)
print(len(unique(prediction)))
train_eval = lambda cluster_labels, d=True: breakdown_clusters_by_ground_truth(prediction, y_train, d)

print(f'The f1 score of fully trained model = {f1_score(y_train, prediction,average="macro")}')
print(f'The v-measure of fully trained model = {train_eval(prediction, d=True)}')